In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.metrics import confusion_matrix, accuracy_score
import skopt
import scipy
from skopt.space import Real
import category_encoders
from sklearn.naive_bayes import GaussianNB
import math

PREDICCION_REAL = False
MAXIMIZAR_HIPERPARAMETROS = False
PARAMETROS = {
    
}



In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_pickle("../Archivos/Arboles_entrenamiento.pkl")
test = pd.read_pickle("../Archivos/Arboles_validacion.pkl")
validacion = pd.read_pickle("../Archivos/Arboles_validacion.pkl")

In [3]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
#fugas = ['ID','Opportunity_Name','Sales_Contract_No','Account_Name','Account_Owner','Opportunity_Owner','Last_Modified_By','Product_Family','Product_Name','ASP','ASP_(converted)']
#'Total_Taxable_Amount'
#columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
#entrenamiento = entrenamiento.drop(columns=fugas)
#test = test.drop(columns=fugas)
#test = test.drop(columns=columnas_fecha)
#entrenamiento = entrenamiento.drop(columns=columnas_fecha)

In [4]:
#FECHAS A DIAS

columnas_fecha = ['Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date',"Year-Month"]
def fecha_a_dias(x):
    for columna in columnas_fecha:
        x[columna] = x[columna].apply(lambda x : (x - pd.to_datetime('01/01/2000', format='%m/%d/%Y')).days)

fecha_a_dias(entrenamiento)
fecha_a_dias(test)

In [5]:
objetivo = (entrenamiento['Stage'] == 1).astype(int)
entrenamiento = entrenamiento.drop(columns=['Stage'])
columnas_category = list(entrenamiento.select_dtypes(include=['category']).columns)
if 'Stage' in columnas_category : columnas_category.remove('Stage')

In [6]:
if not PREDICCION_REAL:
    test_label = (test['Stage'] == 1).astype(int)
    test = test.drop(columns=['Stage'])
    validacion_label = (validacion['Stage'] == 1).astype(int)
    validacion = validacion.drop(columns=['Stage'])

In [7]:
def categoricas_a_numericas(train,test,label,usar_label):
    if (usar_label):
        columnas_object = list(train.select_dtypes(include=['category']).columns)
    else:
        columnas_object = list(test.select_dtypes(include=['category']).columns)
    if 'Stage' in columnas_object : columnas_object.remove('Stage')
    ohe = category_encoders.cat_boost.CatBoostEncoder(cols = columnas_object,return_df = True)
    ohe.fit(train,label)
    if (usar_label):
        columnas = ohe.transform(train,label)
        for columna in columnas_object:
            train[columna] = columnas[columna].copy()
    else:
        columnas = ohe.transform(test)
        for columna in columnas_object:
            test[columna] = columnas[columna].copy()
categoricas_a_numericas(entrenamiento,test,objetivo,False)
categoricas_a_numericas(entrenamiento.copy(),validacion,objetivo,False)
categoricas_a_numericas(entrenamiento,test,objetivo,True)

In [8]:
mean = test["Opportunity_Duration_Ratio"].mean()
test["Opportunity_Duration_Ratio"] = test["Opportunity_Duration_Ratio"].replace(np.NaN,mean)

In [9]:
mean = entrenamiento["Buro_Approved_by_Billing_Country_std"].mean()
entrenamiento["Buro_Approved_by_Billing_Country_std"] = entrenamiento["Buro_Approved_by_Billing_Country_std"].replace(np.NaN,mean)
mean = test["Buro_Approved_by_Billing_Country_std"].mean()
test["Buro_Approved_by_Billing_Country_std"] = test["Buro_Approved_by_Billing_Country_std"].replace(np.NaN,mean)

In [10]:
mean = entrenamiento["Opportunity_Taxable_Rate"].mean()
entrenamiento["Opportunity_Taxable_Rate"] = entrenamiento["Opportunity_Taxable_Rate"].replace(np.NaN,mean)

In [11]:
def limpiar_nan(col):
    mean = entrenamiento[col].mean()
    entrenamiento[col] = entrenamiento[col].replace(np.NaN,mean)
    mean = test[col].mean()
    test[col] = test[col].replace(np.NaN,mean)
    mean = validacion[col].mean()
    validacion[col] = validacion[col].replace(np.NaN,mean)

In [12]:
def limpiar_inf(col):
    entrenamiento[col] = entrenamiento[col].replace(math.inf,np.NaN)
    test[col] = test[col].replace(math.inf,np.NaN)
    validacion[col] = validacion[col].replace(math.inf,np.NaN)

In [13]:
limpiar_nan("ASP_by_Billing_Country_std")
limpiar_nan("Total_Amount_by_Billing_Country_std")
limpiar_nan("Opportunity_Duration_by_Product_Family_std")
limpiar_nan("Total_Amount_by_Product_Family_std")
limpiar_nan("Buro_Approved_by_Product_Family")
limpiar_nan("Territory_Duration")
limpiar_nan("Family_Duration")
limpiar_nan("ASP")
limpiar_nan("Opportunity_TRF")
limpiar_nan("Opportunity_Duration_Ratio")
limpiar_nan("Total_Products_Region_Month_Change")
limpiar_nan("Total_Products_Region_Week_Change")
limpiar_inf("Total_Product_Family_Region_Month_Change")
limpiar_nan("Total_Product_Family_Region_Month_Change")

limpiar_inf("Total_Products_Region_Week_Change") #Baja en 0.001 accuracy
limpiar_nan("Total_Products_Region_Week_Change")

limpiar_inf("Total_Products_Region_Month_Change") #Baja en 0.001 accuracy
limpiar_nan("Total_Products_Region_Month_Change")

In [14]:
entrenamiento = entrenamiento.drop(columns = ["Quote_Expiry_Date","Total_Products_Region_Quarter_Change"])
test = test.drop(columns = ["Quote_Expiry_Date","Total_Products_Region_Quarter_Change"])

In [15]:

for col in entrenamiento.columns:
    print((entrenamiento[col] == math.inf).value_counts())

False    12081
Name: Region, dtype: int64
False    12081
Name: Territory, dtype: int64
False    12081
Name: Pricing, Delivery_Terms_Quote_Appr, dtype: int64
False    12081
Name: Pricing, Delivery_Terms_Approved, dtype: int64
False    12081
Name: Bureaucratic_Code_0_Approval, dtype: int64
False    12081
Name: Bureaucratic_Code_0_Approved, dtype: int64
False    12081
Name: Submitted_for_Approval, dtype: int64
False    12081
Name: Bureaucratic_Code, dtype: int64
False    12081
Name: Account_Created_Date, dtype: int64
False    12081
Name: Source, dtype: int64
False    12081
Name: Billing_Country, dtype: int64
False    12081
Name: Account_Name, dtype: int64
False    12081
Name: Opportunity_Name, dtype: int64
False    12081
Name: Sales_Contract_No, dtype: int64
False    12081
Name: Account_Owner, dtype: int64
False    12081
Name: Opportunity_Owner, dtype: int64
False    12081
Name: Account_Type, dtype: int64
False    12081
Name: Opportunity_Type, dtype: int64
False    12081
Name: Delivery_Te

In [16]:
model = GaussianNB(var_smoothing = 1e-20)

model.fit(entrenamiento,objetivo)
# make the prediction using the resulting model
preds = model.predict(test)
preds

array([0, 1, 1, ..., 1, 1, 1])

In [17]:
accuracy_score(test_label, preds)

0.8843853820598007

In [18]:
model.get_params()

{'priors': None, 'var_smoothing': 1e-20}

In [19]:
preds = model.predict(entrenamiento)
accuracy_score(objetivo, preds)

0.8824600612532075

In [20]:
if (PREDICCION_REAL):
    resultados['Opportunity_ID'] = test['Opportunity_ID']
    resultados['Target'] = pd.Series(preds)
    resultados = resultados.groupby('Opportunity_ID').mean()
    resultados = resultados.reset_index()
    #resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))    
    resultados.to_csv("prediccion.csv", index=False)
    resultados['Target'].value_counts()

In [21]:
if(not MAXIMIZAR_HIPERPARAMETROS): sys.exit()

SystemExit: 

C:\Users\gianb\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.model_selection import StratifiedKFold

# Classifier
bayes_cv_tuner = BayesSearchCV(
estimator = skl.naive_bayes.GaussianNB(),
search_spaces = {
        "var_smoothing" : Real(low=1e-30, high=1e-10, prior='uniform')
},
cv = skl.model_selection.TimeSeriesSplit(),
n_jobs = 30,
n_iter = 50,
verbose = 1,
refit = True,
random_state = 72
)

In [ ]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    

    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))

In [ ]:
resultCAT = bayes_cv_tuner.fit(entrenamiento, objetivo, callback=status_print)

In [ ]:
print(bayes_cv_tuner.best_score_)
print(bayes_cv_tuner.best_params_)

In [ ]:
preds = bayes_cv_tuner.predict(test.values)

In [ ]:
accuracy_score(test_label, preds)